In [1]:
! pip install -qU memorizz

In [2]:
import getpass
import os

# Function to securely get and set environment variables
def set_env_securely(var_name, prompt):
    value = getpass.getpass(prompt)
    os.environ[var_name] = value

In [4]:

set_env_securely("MONGODB_URI", "Enter your MongoDB URI: ")

In [5]:
set_env_securely("OPENAI_API_KEY", "Enter your OpenAI API Key: ")

In [6]:
from memorizz.memory_provider.mongodb.provider import MongoDBConfig, MongoDBProvider

# Create a memory provider
mongodb_config = MongoDBConfig(uri=os.environ["MONGODB_URI"])
memory_provider = MongoDBProvider(mongodb_config)

In [7]:
from memorizz.long_term_memory import KnowledgeBase

# Create a knowledge base
kb = KnowledgeBase(memory_provider)

In [8]:
# Sample knowledge to ingest
company_info = """
Acme Corporation is a fictional company that manufactures everything from portable holes to rocket-powered devices.
Founded in 1952, Acme has been a leading supplier of innovative products for over 70 years.
The company is headquartered in California and has 5,000 employees worldwide.
Their most popular products include the Portable Hole, Rocket Skates, and Giant Rubber Band.
"""

product_info = """
Acme's Portable Hole is a revolutionary product that creates a temporary hole in any surface.
Simply place the Portable Hole on a wall, floor, or ceiling, and it creates a passage to the other side.
The hole can be folded up and carried in a pocket when not in use.
Warning: Do not stack Portable Holes or place them face-to-face, as this may create a rift in space-time.
"""

In [9]:
# Ingest the knowledge
print("Ingesting knowledge...")
company_memory_id = kb.ingest_knowledge(company_info, namespace="company_info")
product_memory_id = kb.ingest_knowledge(product_info, namespace="product_info")
print(f"Created knowledge entries with IDs: {company_memory_id}, {product_memory_id}")

Ingesting knowledge...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Created knowledge entries with IDs: b3be3a5c-0e0f-49f8-b36a-152d97ce8482, 778b2b7c-6045-4b52-a4f8-8fbe2fe0d1f3


In [10]:
from memorizz import MemAgent

# Create a simple agent
agent = MemAgent(
    instruction="You are a helpful assistant that provides information about Acme Corporation and its products.",
    memory_provider=memory_provider
)

In [11]:
# Save the agent to get an agent_id
agent.save()
print(f"Created agent with ID: {agent.agent_id}")


INFO:src.memorizz.memagent:Memagent d3d12ee1-da11-4b35-8245-78c6f1af81dd saved in the memory provider
INFO:src.memorizz.memagent:{
    "model": null,
    "agent_id": "d3d12ee1-da11-4b35-8245-78c6f1af81dd",
    "tools": null,
    "persona": null,
    "instruction": "You are a helpful assistant that provides information about Acme Corporation and its products.",
    "memory_mode": "general",
    "max_steps": 20,
    "memory_ids": [],
    "tool_access": "private",
    "long_term_memory_ids": null
}


Created agent with ID: d3d12ee1-da11-4b35-8245-78c6f1af81dd


In [12]:
# Method 1: Attach knowledge using the KnowledgeBase
kb.attach_to_agent(agent, company_memory_id)
print("Attached company info to agent")

INFO:src.memorizz.memagent:Memagent d3d12ee1-da11-4b35-8245-78c6f1af81dd updated in the memory provider


Attached company info to agent


In [13]:
# Method 2: Attach knowledge using the agent's method
agent.add_long_term_memory(product_info, namespace="additional_product_info")
print("Added product info via agent method")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
ERROR:src.memorizz.memagent:Error adding long-term memory to agent d3d12ee1-da11-4b35-8245-78c6f1af81dd: MemAgent with id d3d12ee1-da11-4b35-8245-78c6f1af81dd already exists


Added product info via agent method


In [14]:
# Retrieve and print all long-term memories associated with the agent
memories = agent.retrieve_long_term_memory()
print(f"\nAgent has {len(memories)} knowledge entries:")
for i, memory in enumerate(memories, 1):
    print(f"{i}. Namespace: {memory.get('namespace')}")
    print(f"   Content: {memory.get('content')[:50]}...")


Agent has 2 knowledge entries:
1. Namespace: company_info
   Content: 
Acme Corporation is a fictional company that manu...
2. Namespace: additional_product_info
   Content: 
Acme's Portable Hole is a revolutionary product t...


In [15]:
# Run some queries to demonstrate knowledge retrieval
print("\nRunning queries with the agent:")

queries = [
    "What is Acme Corporation?",
    "Tell me about the Portable Hole.",
    "When was Acme founded?"
]

for query in queries:
    print(f"\nQuery: {query}")
    response = agent.run(query)
    print(f"Response: {response}")


Running queries with the agent:

Query: What is Acme Corporation?


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Response: Acme Corporation is a well-known fictional company featured in cartoons, comics, and pop culture, often recognized for producing a wide range of imaginative and sometimes outlandish products. Founded in 1952 and headquartered in California, Acme has a legacy of over 70 years and employs around 5,000 people worldwide.

The company is most famous for its innovative and unusual products, many of which are frequently used by characters in classic animated shows. Popular Acme products include:

- Portable Hole: A device that creates a temporary passage through any surface.
- Rocket Skates: High-speed skates powered by rockets.
- Giant Rubber Band: Often used for slingshot-like purposes in comedic scenarios.

Acme Corporation is celebrated for its creative and unexpected solutions, often serving as the go-to brand for characters in need of unusual gadgets or contraptions. While it is a fictional entity, its products and brand have become a staple of popular culture.

Query: Tell me

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Response: Certainly! The Portable Hole is one of Acme Corporation’s most iconic and inventive products.

What is the Portable Hole?
The Portable Hole is a compact, flexible device designed to create an instant opening or passage through virtually any solid surface, including walls, floors, and ceilings. Simply place the Portable Hole on the desired surface, and it forms a functional hole, granting access to the space beyond.

Key Features:

- Portable & Flexible: The hole is lightweight and can be folded up for easy carrying in a pocket or bag.
- Universal Application: Works on multiple surfaces—walls, floors, ceilings, and more.
- Instant Deployment: Creates a passage immediately upon placement.

Safety Information:
- Important: Never stack multiple Portable Holes or place them face-to-face. Doing so can create a dangerous rift in space-time.
- Recommended use is for temporary, non-permanent passages only.

Fun Fact:
The Portable Hole has appeared in countless cartoons and stories as 

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Response: Acme Corporation was founded in 1952.


In [16]:
# Clean up (optional)
print("\nCleaning up...")
agent.delete_long_term_memory(company_memory_id)
agent.delete()

ERROR:src.memorizz.memagent:Error deleting long-term memory b3be3a5c-0e0f-49f8-b36a-152d97ce8482: MemAgent with id d3d12ee1-da11-4b35-8245-78c6f1af81dd already exists
INFO:src.memorizz.memagent:MemAgent d3d12ee1-da11-4b35-8245-78c6f1af81dd deleted from the memory provider



Cleaning up...


True